# Testing web applications

This notebook demonstrates basics of the browser automation and testing of web applications.

Content:

* **[Getting started with Selenium](#Getting-started-with-Selenium)**
* **[Getting started with pytest](#Getting-started-with-pytest)**


Some links:

* https://selenium-python.readthedocs.io/
* https://docs.python.org/3/
* https://www.seleniumhq.org/
* https://docs.pytest.org
* https://www.w3schools.com/xml/xpath_intro.asp

## Getting started with Selenium

Selenium is a software that allows you to automate browser actions. It has bindings for many programming languages. In this workshop we will use python bindings.

In [ ]:
# Import required modules
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [ ]:
# Initialize a webdriver. We will use Firefox.
selenium = webdriver.Remote(desired_capabilities=DesiredCapabilities.FIREFOX)

In [ ]:
# Maximize a browser window
selenium.maximize_window()

Let's try to open a simple microblog application which runs on localhost and let's perform some basic actions there.

In [ ]:
# Open a web page
selenium.get("http://127.0.0.1:5000")

In [ ]:
# Find an element that has attribute 'id' and its value is 'usernaname'
# <input class="form-control" id="username" name="username" required="" type="text" value="">
elem = selenium.find_element_by_id("username")
elem.send_keys("user")

In [ ]:
# <input class="form-control" id="password" name="password" required="" type="password" value="">
elem = selenium.find_element_by_id("password")
elem.send_keys("password")

In [ ]:
# <input class="btn btn-primary" id="submit" name="submit" type="submit" value="Sign In">
elem = selenium.find_element_by_id("submit")
elem.click()

You can locate elements by tags, ids, names, classes and by xpath which is the most versatile way.

In [ ]:
# Logout
elem = selenium.find_element_by_xpath("//nav//a[.='Logout']")
elem.click()

Using selenium you can automate actions for any web page. Try to automate search for seznam.cz, google.com or any other site.

## Getting started with pytest

Pytest is a python testing framework.

In [ ]:
# Import requried stuff to run pytest in a jupyter notebook

import ipytest
import pytest
ipytest.config(rewrite_asserts=True, magics=True)

__file__ = "Testing web applications.ipynb"

In [ ]:
%%run_pytest[clean] -v -qq

def inc(x):
    return x + 1


def test_answer():
    assert inc(3) == 5

### Fixtures

Pytest has a lot of features and concepts. One of the most important and interesting thing is fixtures. The purpose of test fixtures is to provide a fixed baseline upon which tests can reliably and repeatedly execute.

In [ ]:
%%run_pytest[clean] -v -qq

@pytest.fixture
def smtp_connection():
    import smtplib
    return smtplib.SMTP("smtp.gmail.com", 587, timeout=5)

def test_ehlo(smtp_connection):
    response, msg = smtp_connection.ehlo()
    print(response)
    print(msg)
    assert response == 250
    assert 0 # for demo purposes

### Parametrization

In [ ]:
%%run_pytest[clean] -q

from datetime import datetime, timedelta

testdata = [
    (datetime(2001, 12, 12), datetime(2001, 12, 11), timedelta(1)),
    (datetime(2001, 12, 11), datetime(2001, 12, 12), timedelta(1)),
]


@pytest.mark.parametrize("a, b, expected", testdata)
def test_timedistance_v0(a, b, expected):
    diff = a - b
    assert diff == expected

## Testing web applications using selenium and pytest

Using these two libraries we can create automated tests for any web application. Let's test login of our microblog.

In [ ]:
import ipytest
import pytest
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

ipytest.config(rewrite_asserts=True, magics=True)

__file__ = "Testing web applications.ipynb"

@pytest.fixture
def browser():
    browser = webdriver.Remote(desired_capabilities=DesiredCapabilities.FIREFOX)
    browser.maximize_window()
    yield browser
    browser.close()
    
    
@pytest.fixture
def log_in(browser):
    browser.get("http://127.0.0.1:5000")
    elem = browser.find_element_by_id('username')
    elem.send_keys("user")
    elem = browser.find_element_by_id('password')
    elem.send_keys("password")
    elem = browser.find_element_by_id('submit')
    elem.click()

In [ ]:
%%run_pytest[clean] -v -qq

def test_login(browser, log_in):
    assert browser.find_elements_by_xpath("//nav//a[.='Logout']"), "Login wasn't successful - logout button not found"

Let's test post creating and deleting

In [ ]:
%%run_pytest[clean] -v -qq

def test_post_create_delete(browser, log_in):
    elem = browser.find_element_by_id("post")
    elem.send_keys("Test post")
    elem = browser.find_element_by_id("submit")
    elem.click()
    assert browser.find_elements_by_xpath(
        "//div[@class='container']//table//td/span[starts-with(@id, 'post') and .='Test post']")
    elem = browser.find_element_by_xpath(
        "//div[@class='container']//table//td/span[starts-with(@id, 'post') and .='Test post']/../a[.='delete']")
    elem.click()
    assert not browser.find_elements_by_xpath(
        "//div[@class='container']//table//td/span[starts-with(@id, 'post') and .='Test post']")